### Курсовой проект. Создание Телеграмм Бота. https://t.me/Med_test_nlp_bot

In [1]:
!pip install python-telegram-bot --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.6/552.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [2]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 70.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=be84f13e3f0f0c91e8d2e0acc34b18d0060a2fcc1bbebc8f58b1f6e8222a0ff8
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=24ae24e6de902b9f5c41f7856d363c51f8d500833c7b8d3750645152881130d6
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [4]:

!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552446 sha256=b9020b525868084a27088447004f11bbad2be417e000861b9e2eed05cab3c7c2
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [15]:
import numpy as np
import pandas as pd

import os
import annoy
import pickle
import string

from tqdm.notebook import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from telegram.ext  import Updater, CommandHandler, MessageHandler, filters
from telegram import Animation

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
!ls "/content/drive/My Drive"

 all_recepies_inter.csv  'GeekBrains Таблицы'		       TelegramBot
'Colab Notebooks'	 'How to get started with Drive.pdf'   Карьера
 Course_project.ipynb	  lesson_15_LMV_9.ipynb		      'Пробы звука.gdoc'
 CW.ipynb		  NLP				      'Список дел.gsheet'
 English		  Otvety.txt			      'Я начинающий Data Scientist .gdoc'
 final_project_T.ipynb	  prepared_answers.txt
 GeekBrains		  ProductsDataset.csv


In [48]:
products_dataset = pd.read_csv("/content/drive/My Drive/ProductsDataset.csv")
input_file = "/content/drive/My Drive/Otvety.txt"
output_file = "/content/drive/My Drive/prepared_answers.txt"

In [49]:
def preprocess_txt(line):
    """Строковый препроцессинг текстовой строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [50]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [51]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(output_file, "w", encoding="utf8") as fout:
    with open(input_file, "r", encoding="utf8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [52]:
%%time

assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(input_file, "r", encoding="utf8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

CPU times: user 47min 30s, sys: 13 s, total: 47min 43s
Wall time: 49min 5s


In [56]:
sentences[:10]

[['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво'],
 ['советовать', 'сися', 'завести'],
 ['слава', 'бог', 'мужик', 'завести', 'советовать', 'сватать', 'сватать'],
 ['советовать', 'завести', 'любовник', 'муж', 'кошка', 'brприветик'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой'],
 ['вобщий', 'прикалывать', 'тема'],
 ['этобыть',
  'редкость',
  'забавно',
  'знать',
  'фальшивка',
  'прикольно',
  'силиконовый',
  'сиська',
  'налепить',
  'синтетический',
  'волос'],
 ['делать', 'найти', '2', 'миллион', 'рубль']]

In [57]:
sentences = sentences[:50000]

In [58]:
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [59]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [63]:
%%time

index_map = {}
counter = 0

with open(output_file, "r", encoding="utf8") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1
        if counter > 50000:
          break

ft_index.build(10)
ft_index.save('speaker.ann')

0it [00:00, ?it/s]

CPU times: user 5min 29s, sys: 3.33 s, total: 5min 32s
Wall time: 5min 37s


True

In [64]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')

True

In [65]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[2943, 65591]

In [66]:
products_dataset['text'] = products_dataset['title'] + " " + products_dataset["descrirption"]
products_dataset['text'] = products_dataset['text'].apply(lambda x: preprocess_txt(str(x)))
products_dataset.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [67]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [68]:
idxs = set(np.random.randint(0, len(index_map), len(products_dataset)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in products_dataset['text'].values]

In [69]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [70]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2,
                                                    stratify=labels, random_state=13)

In [71]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [72]:
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.977216849696771

In [73]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [74]:
tfidf_vect = TfidfVectorizer().fit(X_train)

In [75]:
np.mean(tfidf_vect.idf_)

10.772501644681196

In [76]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'предрекать': 9.022999323369284,
 'заварушка': 6.959306138657587,
 'платье': 10.186150133174966,
 '4446': 11.10244086504912,
 'одетый': 11.10244086504912,
 'побрызгать': 11.10244086504912,
 'антимоль': 11.10244086504912,
 'забывать': 11.10244086504912,
 'пылесосить': 11.10244086504912,
 'хранение': 11.10244086504912,
 '17': 11.10244086504912,
 'называть': 11.10244086504912,
 'мода': 11.10244086504912,
 '16': 11.10244086504912,
 'сделать': 11.10244086504912,
 'немец': 11.10244086504912,
 'изменить': 11.10244086504912,
 'скин': 11.10244086504912,
 'появиться': 11.10244086504912,
 'повторять': 11.10244086504912,
 'выйти': 11.10244086504912,
 'counterstrike': 11.10244086504912,
 'online': 11.10244086504912,
 'counterstrikemasan': 11.10244086504912,
 'влиять': 11.10244086504912,
 'внутренний': 11.10244086504912,
 'мир': 11.10244086504912,
 'виктория': 11.10244086504912,
 'романец': 11.10244086504912,
 'проект': 10.696975756940956,
 'дом': 11.10244086504912,
 'прийти': 11.10244086504912,
 '

In [78]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(products_dataset))):
    n_ft = 0
    index_map_shop[counter] = (products_dataset.loc[i, "title"], products_dataset.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in products_dataset.loc[i, "text"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1
    if counter > 50000:
          break

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

  0%|          | 0/35548 [00:00<?, ?it/s]

True

In [79]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [80]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([727], [1.4142135381698608])

In [81]:
!pip install telegram

  Preparing metadata (setup.py) ... done
  Created wheel for telegram: filename=telegram-0.0.1-py3-none-any.whl size=1293 sha256=634c02f456cfe6edab889bed1e4036632645701ff3055a244dc72ecf312919b3
  Stored in directory: /root/.cache/pip/wheels/f3/f2/16/dc6d6c6c7e0422206fec62833039a2b4082de3b85e55d32b52
Successfully built telegram


In [82]:
!pip uninstall tornado
!pip install tornado==4.5.3

Found existing installation: tornado 6.3.2
Uninstalling tornado-6.3.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tornado-6.3.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tornado/*
Proceed (Y/n)? Y
  Successfully uninstalled tornado-6.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tornado: filename=tornado-4.5.3-cp310-cp310-linux_x86_64.whl size=427990 sha256=a161a4a8bf07f1e9c5ab10213b4e4fe9a4ed3ad8fd87fead6c86383c298a5caf
  Stored in directory: /root/.cache/pip/wheels/7a/fc/0f/8294151792cdc6cd7041df1467dbb536b4cff20d161d58501c
Successfully built tornado
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 3.3.2 requires tornado>=5.1, but you have tornado 4.5.3 which is incompatible.
distributed 2023.8.1 requires tornado>=6.0.4, but

In [83]:

!pip uninstall telegram==0.0.1

Found existing installation: telegram 0.0.1
Uninstalling telegram-0.0.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/telegram-0.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/telegram/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/telegram/__main__.py
    /usr/local/lib/python3.10/dist-packages/telegram/_bot.py
    /usr/local/lib/python3.10/dist-packages/telegram/_botcommand.py
    /usr/local/lib/python3.10/dist-packages/telegram/_botcommandscope.py
    /usr/local/lib/python3.10/dist-packages/telegram/_botdescription.py
    /usr/local/lib/python3.10/dist-packages/telegram/_botname.py
    /usr/local/lib/python3.10/dist-packages/telegram/_callbackquery.py
    /usr/local/lib/python3.10/dist-packages/telegram/_chat.py
    /usr/local/lib/python3.10/dist-packages/telegram/_chatadministratorrights.py
    /usr/local/lib/python3.10/dist-packages/telegram/_chatinvitelink.py
    /usr/local/lib/python3.10/dist-packages/teleg

In [84]:
start_msg = 'Привет. Я бот.  Чем могу помочь?'

In [150]:
pip install python-telegram-bot==13.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.2/495.2 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: python-telegram-bot
    Found existing installation: python-telegram-bot 13.13
    Uninstalling python-telegram-bot-13.13:
      Successfully uninstalled python-telegram-bot-13.13


In [135]:
from telegram.ext import Application, CommandHandler, MessageHandler, filters

In [136]:
from telegram.ext  import Updater, CommandHandler, MessageHandler, filters

In [99]:
app =Application.builder().token("6802447819:AAG6j5cSgH1-uUPIhDQ6UxEk9Yk4PAGKTD8").build()

In [100]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=start_msg)

In [101]:
def textMessage(bot, update):

    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)

    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        bot.send_message(chat_id=update.message.chat_id, text="Чаво ?")
        return
    bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

In [164]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(filters, textMessage)
app.add_handler(start_command_handler)
app.add_handler(text_message_handler)
app.start()
app.stop()

<ipython-input-164-5e2f3f509662>:5: RuntimeWarning: coroutine 'Application.start' was never awaited
  app.start()


<coroutine object Application.stop at 0x7db6ed020200>